In [ ]:
import glob
import os

import anndata as ad
import numpy as np
import pandas as pd
import scanpy as sc


In [ ]:
# Allocate the base directory of the data
base_dir = "/scratch/bvdberg/SoloTE/"

# Use glob to find directories matching the pattern
directories = glob.glob(
    os.path.join(base_dir, "run_*/", "*_SoloTE_output/", "*_locustes_MATRIX")
)
anndata = {}


In [ ]:
for directory in directories:
    # Set the wildcard per directory
    wildcard = os.path.basename(directory)
    identifier = str(wildcard.split("_locustes_MATRIX")[0])
    
    # Skip directories with an empty identifier
    if not identifier:
        continue

    # Check if the required files exist in the directory
    matrix_file = os.path.join(directory, "matrix.mtx")
    barcodes_file = os.path.join(directory, "barcodes.tsv")
    features_file = os.path.join(directory, "features.tsv")

    if (
        os.path.exists(matrix_file)
        and os.path.exists(barcodes_file)
        and os.path.exists(features_file)
    ):
        # Create a new AnnData object for each directory
        adata = sc.AnnData()

        # Read the matrix using scanpy
        adata = sc.read_mtx(matrix_file)
        adata = adata.transpose()

        # Read barcodes and features using pandas
        barcodes = pd.read_csv(barcodes_file, sep="\t", header=None, names=["barcode"])
        features = pd.read_csv(features_file, sep="\t", header=None, names=["gene_name"])

        # Set obs_names and var_names
        adata.obs_names = barcodes["barcode"]
        adata.var_names = features["gene_name"]

        anndata[identifier] = adata

    else:
        print(f"Required files not found in directory: {directory}")


In [ ]:
# Combine the adata sets to one data set with the Concatenate function, we use 'outer' to preserve as much data as possible. Missing variablles will become NaN values
combined_adata = ad.concat(anndata, label="dataset_origin", join="outer")


In [ ]:
# Modify the data to make it comparable to a whitelisted h5ad formatted file, by manipulating the 'dataset origin' column
combined_adata.obs.index = combined_adata.obs['dataset_origin'].str[1:] + '_' + combined_adata.obs.index


In [ ]:
combined_adata.write_h5ad("combined_bonemarrow_sets.h5ad")


In [ ]:
whitelist_anndata = ad.read_h5ad("bonemarrow_m.h5ad")


In [ ]:
# Subset the anndata based on the whitelist
filtered_anndata = combined_adata[combined_adata.obs.index.isin(whitelist_anndata.obs.index)]


In [ ]:
# Copy metadata from the whitelist to the filtered dataset
filtered_anndata.obs = whitelist_anndata.obs


In [ ]:
# Initialize the 'TE_type' column
filtered_anndata.var['te_type'] = 'GENE'

# Create a mask based on the structure you want to match
mask = filtered_anndata.var_names.str.count(':') > 1

# Apply the structure-based modification to matching entries
matching_entries = filtered_anndata.var_names[mask]
matching_entries = matching_entries.astype(str).str.split(':').str[2]
matching_entries = matching_entries.str.split('|').str[0]

# Update the 'te_type' column for matching entries
filtered_anndata.var['te_type'][mask] = matching_entries


In [ ]:
filtered_anndata.write_h5ad("whitelisted_bonemarrow.h5ad")
